# Comparison with article

In [1]:
from coupled_cluster.schemes.ccd_sparse import CoupledClusterDoublesSparse
from coupled_cluster.hartree_fock.scf_rhf import scf_rhf
from coupled_cluster.matrix_elements.generate_matrices import (
    get_one_body_elements, get_coulomb_elements,
    get_antisymmetrized_elements, add_spin_to_one_body_elements,
    get_one_body_elements_spin
)
from coupled_cluster.hartree_fock.basis_transformation import (
    transform_one_body_elements, transform_two_body_elements
)

import numpy as np
import time
import os

In [2]:
file_path = os.path.join("..", "dat")
filename = os.path.join(file_path, "coulomb_{0}.pkl")

omega = 1.0
l = 110
n = 20
theta = 0.8

filename = filename.format(l)

print ("""
w = {0},
l = {1},
n = {2},
theta = {3},
filename = {4}
""".format(omega, l, n, theta, filename))


w = 1.0,
l = 110,
n = 20,
theta = 0.8,
filename = ../dat/coulomb_110.pkl



In [3]:
h = omega * get_one_body_elements(l)
t0 = time.time()
u = np.sqrt(omega) * get_coulomb_elements(l, filename=filename)
t1 = time.time()
print ("Time spent creating Coulomb elements: {0} sec".format(t1 - t0))

t0 = time.time()
c, energy = scf_rhf(h.todense(), u, np.eye(l//2), n//2)
t1 = time.time()
print ("Time spent in SCF RHF: {0} sec".format(t1 - t0))

print ("\tRHF Energy: {0}".format(energy))

hi = transform_one_body_elements(h, c)
t0 = time.time()
oi = transform_two_body_elements(u, c)
t1 = time.time()
print ("Time spent transforming two body elements: {0} sec".format(t1 - t0))

_h = add_spin_to_one_body_elements(hi, l)
t0 = time.time()
_u = get_antisymmetrized_elements(l, oi=oi)
t1 = time.time()
print ("Time spent antisymmetrizing two body elements: {0} sec".format(t1 - t0))

t0 = time.time()
ccd_hf = CoupledClusterDoublesSparse(_h, _u, n)
t1 = time.time()
print ("Time spent setting up CCD code with HF basis: {0} sec".format(t1 - t0))
t0 = time.time()
energy, iterations = ccd_hf.compute_energy(tol=1e-4, theta=theta)
t1 = time.time()
print ("Time spent computing CCD energy with HF basis: {0} sec".format(t1 - t0))
print ("\tCCD (HF) Energy: {0}\n\tIterations: {1}\n\tSecond/iteration: {2}".format(energy, iterations, (t1 - t0)/iterations))

"""
__h = omega * get_one_body_elements_spin(l)
t0 = time.time()
__u = np.sqrt(omega) * get_antisymmetrized_elements(l, filename=filename)
t1 = time.time()
print ("Time spent getting antisymmetric two body elements: {0} sec".format(t1 - t0))

t0 = time.time()
ccd = CoupledClusterDoublesSparse(__h, __u, n)
t1 = time.time()
print ("Time spent setting up CCD code with HO basis: {0} sec".format(t1 - t0))
t0 = time.time()
energy, iterations = ccd.compute_energy(theta=theta)
t1 = time.time()
print ("Time spent computing CCD energy with HO basis: {0} sec".format(t1 - t0))
print ("\tCCD Energy: {0}\n\tIterations: {1}\n\tSecond/iteration: {2}".format(energy, iterations, (t1 - t0)/iterations))
"""

Time spent creating Coulomb elements: 0.013513803482055664 sec
Time spent in SCF RHF: 3.2675211429595947 sec
	RHF Energy: 159.0375866310746
Time spent transforming two body elements: 1.675553321838379 sec
Time spent antisymmetrizing two body elements: 1.5380547046661377 sec
Time spent setting up CCD code with HF basis: 1.584740400314331 sec
157.38091434320464
157.4211783938602
157.445040309221
157.4584574309236
157.46521660528097
157.46776879292753
Time spent computing CCD energy with HF basis: 12.772682189941406 sec
	CCD (HF) Energy: 157.4677157460408
	Iterations: 6
	Second/iteration: 2.1287803649902344


'\n__h = omega * get_one_body_elements_spin(l)\nt0 = time.time()\n__u = np.sqrt(omega) * get_antisymmetrized_elements(l, filename=filename)\nt1 = time.time()\nprint ("Time spent getting antisymmetric two body elements: {0} sec".format(t1 - t0))\n\nt0 = time.time()\nccd = CoupledClusterDoublesSparse(__h, __u, n)\nt1 = time.time()\nprint ("Time spent setting up CCD code with HO basis: {0} sec".format(t1 - t0))\nt0 = time.time()\nenergy, iterations = ccd.compute_energy(theta=theta)\nt1 = time.time()\nprint ("Time spent computing CCD energy with HO basis: {0} sec".format(t1 - t0))\nprint ("\tCCD Energy: {0}\n\tIterations: {1}\n\tSecond/iteration: {2}".format(energy, iterations, (t1 - t0)/iterations))\n'

In [4]:
print (h.density)
print (u.density)
print (_u.density)
print (_h.density)
print (__u.density)
print (__h.density)

0.01818181818181818
0.04608067754934772
0.017114787241308653
0.032231404958677684


NameError: name '__u' is not defined